In [ ]:
# 코랩 환경 한글 글꼴 설치, 설치 후 런타임 재시작
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,214 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121658 files and dire

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt


#plt.rc('font', family='NanumBarunGothic')

# [1] 데이터 로드, 데이터 살펴보기

- 일시
- 최고기온
- 최저기온
- 일교차
- 강수량
- 평균습도
- 평균풍속
- 일조합
- 일사합
- 일조율
- 평균기온(Target)

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Dacon/23_서울시평균기온예측/data/train.csv', encoding = 'utf-8')
sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_서울시평균기온예측/data/sample_submission.csv', encoding = 'utf-8')

In [4]:
train.head()

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB


In [6]:
data = train.copy()

# [2] 전처리

- 타겟 분리

In [7]:
target = data['평균기온']

data.drop(['평균기온'], axis = 1, inplace = True)

## 2.1 기본 칼럼 분할

- 연월일 분할

In [8]:
data['연'] = pd.to_datetime(data['일시']).dt.year
data['월'] = pd.to_datetime(data['일시']).dt.month
data['일'] = pd.to_datetime(data['일시']).dt.day

In [9]:
data.drop(['일시'], axis = 1, inplace = True)

- 계절, 분기

In [10]:
def to_season( x ):
    if x in [12, 1, 2]:
        return 0 # 겨울
    elif x in [3, 4, 5]:
        return 1 # 봄
    elif x in [6, 7, 8]:
        return 2 # 여름
    else:
        return 3 # 가을
    pass

data['계절'] = data['월'].apply(lambda x : to_season(x))

# def to_quarter( x ):
#     if x in [1,2,3]:
#         return 0 # 1분기
#     elif x in [4,5,6]:
#         return 1 # 2분기
#     elif x in [7, 8, 9]:
#         return 2 # 3분기
#     else:
#         return 3 # 4분기
#     pass

# data['분기'] = data['월'].apply(lambda x : to_quarter(x))
data.head(2)

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절
0,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,1960,1,1,0
1,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,1960,1,2,0


## 2.2 NA 처리

- 칼럼별로 다른 방식으로 접근한다
    - 최고기온, 최저기온, 평균습도, 평균풍속 : 앞으로 채우고, 뒤로 채우고 (가까운 값으로 대치)
    - 일교차 : 최고기온 - 최저기온 다시 계산해보고, 없으면 역시 앞으로 채우고 뒤로 채우고
    - 강수량 : 우선 0으로 채운다.
    - 일조합, 일사합, 일조율 : 해당 연도의 계절별 평균으로 대체


- 최저기온, 최고기온, 평균습도, 평균풍속

In [11]:
for column in ['최저기온', '최고기온', '평균습도', '평균풍속']:
    data[column].fillna(method='ffill', inplace=True)
    data[column].fillna(method='bfill', inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   최고기온    23011 non-null  float64
 1   최저기온    23011 non-null  float64
 2   일교차     23007 non-null  float64
 3   강수량     9150 non-null   float64
 4   평균습도    23011 non-null  float64
 5   평균풍속    23011 non-null  float64
 6   일조합     22893 non-null  float64
 7   일사합     18149 non-null  float64
 8   일조율     22645 non-null  float64
 9   연       23011 non-null  int64  
 10  월       23011 non-null  int64  
 11  일       23011 non-null  int64  
 12  계절      23011 non-null  int64  
dtypes: float64(9), int64(4)
memory usage: 2.3 MB


- 일교차

In [12]:
data.loc[data['일교차'].isna(), '일교차'] = data['최고기온'] - data['최저기온']

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   최고기온    23011 non-null  float64
 1   최저기온    23011 non-null  float64
 2   일교차     23011 non-null  float64
 3   강수량     9150 non-null   float64
 4   평균습도    23011 non-null  float64
 5   평균풍속    23011 non-null  float64
 6   일조합     22893 non-null  float64
 7   일사합     18149 non-null  float64
 8   일조율     22645 non-null  float64
 9   연       23011 non-null  int64  
 10  월       23011 non-null  int64  
 11  일       23011 non-null  int64  
 12  계절      23011 non-null  int64  
dtypes: float64(9), int64(4)
memory usage: 2.3 MB


- 강수량

In [13]:
data['강수량'] = data['강수량'].fillna(0)

In [14]:
data.tail()

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절
23006,3.3,-7.3,10.6,0.0,69.8,1.8,8.8,10.25,91.7,2022,12,27,0
23007,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.86,90.6,2022,12,28,0
23008,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.88,93.8,2022,12,29,0
23009,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.84,82.3,2022,12,30,0
23010,2.1,-5.1,7.2,0.0,65.5,1.4,1.1,4.16,11.5,2022,12,31,0


- 일조합, 일사합, 일조율

In [15]:
for column in ['일조합', '일사합', '일조율']:
    data[column] = data.groupby(['월'])[column].transform(lambda x: x.fillna(x.mean()))

data.head()

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절
0,2.2,-5.2,7.4,0.0,68.3,1.7,6.7,7.409729,54.474246,1960,1,1,0
1,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,7.409729,54.474246,1960,1,2,0
2,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,7.409729,54.474246,1960,1,3,0
3,10.8,1.2,9.6,0.0,79.7,4.4,2.6,7.409729,54.474246,1960,1,4,0
4,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,7.409729,54.474246,1960,1,5,0


In [16]:
data

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절
0,2.2,-5.2,7.4,0.0,68.3,1.7,6.7,7.409729,54.474246,1960,1,1,0
1,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,7.409729,54.474246,1960,1,2,0
2,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,7.409729,54.474246,1960,1,3,0
3,10.8,1.2,9.6,0.0,79.7,4.4,2.6,7.409729,54.474246,1960,1,4,0
4,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,7.409729,54.474246,1960,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,3.3,-7.3,10.6,0.0,69.8,1.8,8.8,10.250000,91.700000,2022,12,27,0
23007,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.860000,90.600000,2022,12,28,0
23008,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.880000,93.800000,2022,12,29,0
23009,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.840000,82.300000,2022,12,30,0


## 2-3. 스케일링

In [17]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.iloc[:, :9])

data_scaled = pd.DataFrame(data_scaled, columns=data.columns[:9])
data_scaled.head()

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율
0,0.296992,0.297030,0.344086,0.000000,0.615385,0.207792,0.489051,0.221318,0.56217
1,0.278195,0.289109,0.311828,0.001202,0.852259,0.155844,0.000000,0.221318,0.56217
2,0.419173,0.358416,0.526882,0.000000,0.774115,0.376623,0.000000,0.221318,0.56217
3,0.458647,0.423762,0.462366,0.000000,0.754579,0.558442,0.189781,0.221318,0.56217
4,0.280075,0.237624,0.456989,0.000000,0.318681,0.649351,0.598540,0.221318,0.56217


In [18]:
data.iloc[:, :9] = data_scaled

In [19]:
data

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절
0,0.296992,0.297030,0.344086,0.000000,0.615385,0.207792,0.489051,0.221318,0.562170,1960,1,1,0
1,0.278195,0.289109,0.311828,0.001202,0.852259,0.155844,0.000000,0.221318,0.562170,1960,1,2,0
2,0.419173,0.358416,0.526882,0.000000,0.774115,0.376623,0.000000,0.221318,0.562170,1960,1,3,0
3,0.458647,0.423762,0.462366,0.000000,0.754579,0.558442,0.189781,0.221318,0.562170,1960,1,4,0
4,0.280075,0.237624,0.456989,0.000000,0.318681,0.649351,0.598540,0.221318,0.562170,1960,1,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,0.317669,0.255446,0.516129,0.000000,0.633700,0.220779,0.642336,0.306153,0.946336,2022,12,27,0
23007,0.257519,0.281188,0.274194,0.000300,0.490842,0.311688,0.635036,0.324373,0.934985,2022,12,28,0
23008,0.295113,0.245545,0.478495,0.000000,0.468864,0.207792,0.656934,0.324970,0.968008,2022,12,29,0
23009,0.298872,0.312871,0.306452,0.000000,0.582418,0.233766,0.576642,0.323775,0.849329,2022,12,30,0


In [20]:
data['target'] = target
data

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절,target
0,0.296992,0.297030,0.344086,0.000000,0.615385,0.207792,0.489051,0.221318,0.562170,1960,1,1,0,-1.6
1,0.278195,0.289109,0.311828,0.001202,0.852259,0.155844,0.000000,0.221318,0.562170,1960,1,2,0,-1.9
2,0.419173,0.358416,0.526882,0.000000,0.774115,0.376623,0.000000,0.221318,0.562170,1960,1,3,0,4.0
3,0.458647,0.423762,0.462366,0.000000,0.754579,0.558442,0.189781,0.221318,0.562170,1960,1,4,0,7.5
4,0.280075,0.237624,0.456989,0.000000,0.318681,0.649351,0.598540,0.221318,0.562170,1960,1,5,0,-4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,0.317669,0.255446,0.516129,0.000000,0.633700,0.220779,0.642336,0.306153,0.946336,2022,12,27,0,-2.6
23007,0.257519,0.281188,0.274194,0.000300,0.490842,0.311688,0.635036,0.324373,0.934985,2022,12,28,0,-3.3
23008,0.295113,0.245545,0.478495,0.000000,0.468864,0.207792,0.656934,0.324970,0.968008,2022,12,29,0,-2.9
23009,0.298872,0.312871,0.306452,0.000000,0.582418,0.233766,0.576642,0.323775,0.849329,2022,12,30,0,-1.8


# [3] 모델링

## NeuralProphet

In [21]:
!pip install neuralprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.2/378.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.2/185.2 kB 23.6 

In [22]:
data['ds'] = train.일시
data = data.rename(columns={'target': 'y'})

In [23]:
data

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,연,월,일,계절,y,ds
0,0.296992,0.297030,0.344086,0.000000,0.615385,0.207792,0.489051,0.221318,0.562170,1960,1,1,0,-1.6,1960-01-01
1,0.278195,0.289109,0.311828,0.001202,0.852259,0.155844,0.000000,0.221318,0.562170,1960,1,2,0,-1.9,1960-01-02
2,0.419173,0.358416,0.526882,0.000000,0.774115,0.376623,0.000000,0.221318,0.562170,1960,1,3,0,4.0,1960-01-03
3,0.458647,0.423762,0.462366,0.000000,0.754579,0.558442,0.189781,0.221318,0.562170,1960,1,4,0,7.5,1960-01-04
4,0.280075,0.237624,0.456989,0.000000,0.318681,0.649351,0.598540,0.221318,0.562170,1960,1,5,0,-4.6,1960-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,0.317669,0.255446,0.516129,0.000000,0.633700,0.220779,0.642336,0.306153,0.946336,2022,12,27,0,-2.6,2022-12-27
23007,0.257519,0.281188,0.274194,0.000300,0.490842,0.311688,0.635036,0.324373,0.934985,2022,12,28,0,-3.3,2022-12-28
23008,0.295113,0.245545,0.478495,0.000000,0.468864,0.207792,0.656934,0.324970,0.968008,2022,12,29,0,-2.9,2022-12-29
23009,0.298872,0.312871,0.306452,0.000000,0.582418,0.233766,0.576642,0.323775,0.849329,2022,12,30,0,-1.8,2022-12-30


In [24]:
train_df = data[['ds','y']]

In [ ]:
from neuralprophet import NeuralProphet

prophet = NeuralProphet()

prophet.fit(train_df)

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.996% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.996% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO:NP.df_utils:Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to gl

Finding best initial lr:   0%|          | 0/259 [00:00<?, ?it/s]

In [ ]:
# from neuralprophet import NeuralProphet

# prophet = NeuralProphet(
#     n_changepoints=2170,  # changepoints의 수 설정
#     ar_layers = 939,
#     yearly_seasonality=True,  # 연간 계절성 사용 여부
#     loss_func='rmse',  # MAE 손실 함수 설정
# )
# prophet.fit(train_df)

In [ ]:
#모델 예측

future_data = prophet.make_future_dataframe(df = train_df,periods = 358)

forecast_data = prophet.predict(future_data)

In [ ]:
forecast_data

,ds,y,yhat1,trend,season_yearly,season_weekly
0,2023-01-01,None,-1.141409,13.808849,-14.964678,0.014420
1,2023-01-02,None,-1.224993,13.809099,-15.045642,0.011550
2,2023-01-03,None,-1.329307,13.809345,-15.119699,-0.018952
3,2023-01-04,None,-1.405207,13.809595,-15.186816,-0.027984
4,2023-01-05,None,-1.455575,13.809841,-15.246946,-0.018469
...,...,...,...,...,...,...
353,2023-12-20,None,0.426647,13.896286,-13.441655,-0.027984
354,2023-12-21,None,0.277349,13.896536,-13.600717,-0.018469
355,2023-12-22,None,0.132099,13.896784,-13.753953,-0.010733
356,2023-12-23,None,0.045950,13.897032,-13.901251,0.050169


In [ ]:
sub['평균기온'] = forecast_data['yhat1']
sub

,일시,평균기온
0,2023-01-01,-1.141409
1,2023-01-02,-1.224993
2,2023-01-03,-1.329307
3,2023-01-04,-1.405207
4,2023-01-05,-1.455575
...,...,...
353,2023-12-20,0.426647
354,2023-12-21,0.277349
355,2023-12-22,0.132099
356,2023-12-23,0.045950


In [ ]:
sub.to_csv('sub_2.csv', index = False)

## 3개년 단순 평균

In [ ]:
train['일시'].str[5:]

0        01-01
1        01-02
2        01-03
3        01-04
4        01-05
         ...  
23006    12-27
23007    12-28
23008    12-29
23009    12-30
23010    12-31
Name: 일시, Length: 23011, dtype: object

In [ ]:
data_tmp = data[data['연'] >= 2020]
data_tmp['월일'] = train['일시'].str[5:]

WARNING - (py.warnings._showwarnmsg) - <ipython-input-155-7cca0b1c012f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['월일'] = train['일시'].str[5:]

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['월일'] = train['일시'].str[5:]



In [ ]:
tmp = data_tmp.groupby(['월일']).mean()['y'].reset_index()

WARNING - (py.warnings._showwarnmsg) - <ipython-input-158-cde2866fcdef>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tmp = data_tmp.groupby(['월일']).mean()['y'].reset_index()

  tmp = data_tmp.groupby(['월일']).mean()['y'].reset_index()



In [ ]:
tmp

,월일,y
0,01-01,-3.566667
1,01-02,-1.766667
2,01-03,-2.533333
3,01-04,-1.600000
4,01-05,-2.333333
...,...,...
361,12-27,-1.466667
362,12-28,-0.233333
363,12-29,-0.800000
364,12-30,-5.533333


In [ ]:
sub['월일'] = sub['일시'].str[5:]

In [ ]:
sub_final = sub.merge(tmp, how = 'left', on = '월일')
sub_final

,일시,평균기온,월일,y
0,2023-01-01,-1.141409,01-01,-3.566667
1,2023-01-02,-1.224993,01-02,-1.766667
2,2023-01-03,-1.329307,01-03,-2.533333
3,2023-01-04,-1.405207,01-04,-1.600000
4,2023-01-05,-1.455575,01-05,-2.333333
...,...,...,...,...
353,2023-12-20,0.426647,12-20,-0.566667
354,2023-12-21,0.277349,12-21,1.366667
355,2023-12-22,0.132099,12-22,-1.200000
356,2023-12-23,0.045950,12-23,-1.533333


In [ ]:
sub_final['평균기온'] = sub_final['y']

In [ ]:
sub_final = sub_final[['일시', '평균기온']]

In [ ]:
sub_final.to_csv('sub_3.csv', index = False)

## LGBM : 22년도 예측

- train, test 분할

In [ ]:
test = data.iloc[22646: 22646 + 358, :]
train = data.iloc[: 22646, :]

In [ ]:
y_train = train.target
y_test = test.target

train.drop(['target'], axis = 1, inplace = True)
test.drop(['target'], axis = 1, inplace = True)

<ipython-input-84-70a0064cacb3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(['target'], axis = 1, inplace = True)
<ipython-input-84-70a0064cacb3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(['target'], axis = 1, inplace = True)


- train, valid 분할

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y_train, test_size=0.2, random_state=42)

- 모델 생성

In [ ]:
model = LGBMRegressor()

- 학습

In [ ]:
model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='rmse')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004532 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2032
[LightGBM] [Info] Number of data points in the train set: 18116, number of used features: 13
[LightGBM] [Info] Start training from score 12.451253


LGBMRegressor()

- Validation 확인

In [ ]:
y_pred = model.predict(X_valid)
rmse_score = sqrt(mean_squared_error(y_valid, y_pred))

In [ ]:
rmse_score # validation Score

- 예측

In [ ]:
y_pred = model.predict(test)
y_pred

array([ -4.32062044,  -1.21207448,  -3.05316982,  -2.695726  ,
        -2.56214659,  -1.84351361,  -1.11622923,  -0.30525879,
         1.60565647,   0.40041551,  -5.72719576,  -7.90883519,
        -6.29352277,  -4.66512384,  -1.61458272,  -2.68162592,
        -4.21626252,  -5.52768368,  -5.03782725,  -4.49379903,
        -3.18895129,   1.0781188 ,   2.51391441,   4.60980832,
         3.31396576,   1.85340034,  -0.90971223,  -2.13012084,
        -3.47704043,  -3.17433994,  -1.24480924,  -0.75803225,
        -3.2515832 ,  -3.5740821 ,  -4.66431792,  -6.12525277,
        -3.43738834,  -2.41325903,   0.35287315,   0.91083233,
         1.29287106,   2.75503498,   4.26318639,   5.11288017,
         3.2845574 ,  -4.76776125,  -7.24549847,  -5.77589029,
        -2.10885945,  -0.78091256,  -4.67411398,  -2.04555168,
        -3.89324645,  -5.38099008,  -3.03561998,   2.26863229,
         5.11016066,   3.43911904,   6.60538868,   4.67049505,
         3.13467117,   6.24764058,   7.7564986 ,   2.54

In [ ]:
sub['평균기온'] = y_pred
sub

,일시,평균기온
0,2023-01-01,-4.320620
1,2023-01-02,-1.212074
2,2023-01-03,-3.053170
3,2023-01-04,-2.695726
4,2023-01-05,-2.562147
...,...,...
353,2023-12-20,-3.661804
354,2023-12-21,0.190363
355,2023-12-22,-7.404140
356,2023-12-23,-12.173365


In [ ]:
sub.to_csv('sub_1.csv', index = False)